In [3]:
from wandb.integration.keras import WandbMetricsLogger
from mlp_explainer import explainer, metrics
from scikeras.wrappers import KerasClassifier
from pgmpy.utils import get_example_model
from tensorflow import keras
import mlp_explainer
import numpy as np
import sklearn
import wandb
import copy

In [4]:
wandb.login()

True

In [7]:
from ucimlrepo import fetch_ucirepo 
  
car_evaluation = fetch_ucirepo(id = 19) 

X = car_evaluation.data.features.to_numpy()
y = car_evaluation.data.targets.to_numpy()

In [8]:
def build_model(input_shape, classes, n_layers = 5, n_nodes_l = 2, n_nodes_u = 10, optimizer = 'adam', learning_rate = 0.001):
    
    model = keras.Sequential()
    model.add(keras.Input(shape = (input_shape,)))
    for i in range(n_layers):
        model.add(keras.layers.Dense(np.random.choice([i for i in range(n_nodes_l, n_nodes_u)]), activation = np.random.choice(['relu', 'sigmoid'])))
    model.add(keras.layers.Dense(classes, activation = 'softmax'))

    if optimizer == 'sgd':
        optim = keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == 'adam':
        optim = keras.optimizers.Adam(learning_rate = learning_rate)
    
    model.compile(optimizer = optim, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [26]:
n_layers = 5
n_nodes_l = 5
n_nodes_u = 50
optimizer = 'adam'
learning_rate = 0.01
epochs = 25
batch_size = 64

In [34]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, test_size = 0.1, random_state = 42, shuffle = True
)

In [35]:
feature_encoder = sklearn.preprocessing.OrdinalEncoder()
feature_encoder.fit(X_train)

target_encoder = sklearn.preprocessing.LabelEncoder()
target_encoder.fit(y_train.squeeze())

LabelEncoder()

In [36]:
X_train_proc = feature_encoder.transform(X_train)
y_train_proc = target_encoder.transform(y_train)

/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [37]:
n_classes = len(np.unique(y))
input_shape = len(X[0])

In [38]:
wandb.init(entity = "computer-vision-wits", project = "MLP-E", name = "")
    
model = build_model(
    input_shape,
    n_classes,
    n_layers = n_layers, 
    n_nodes_l = n_nodes_l,
    n_nodes_u = n_nodes_u, 
    optimizer = optimizer, 
    learning_rate = learning_rate
)
    
config = wandb.config
config.n_layers = n_layers
config.n_nodes_l = n_nodes_l
config.n_nodes_u = n_nodes_u
config.optimizer = optimizer
config.epochs = epochs
config.batch_size = batch_size
config.learning_rate = learning_rate
    
model.fit(X_train_proc, y_train_proc, epochs = epochs, batch_size = batch_size, validation_split = 0.2, callbacks = [WandbMetricsLogger()])
model.summary()

wandb.finish()

/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


Epoch 1/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7050 - loss: 0.8051 - val_accuracy: 0.6656 - val_loss: 0.8037
Epoch 2/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7154 - loss: 0.6485 - val_accuracy: 0.6881 - val_loss: 0.7176
Epoch 3/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7170 - loss: 0.5985 - val_accuracy: 0.6688 - val_loss: 0.6754
Epoch 4/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7154 - loss: 0.5728 - val_accuracy: 0.6849 - val_loss: 0.6581
Epoch 5/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7404 - loss: 0.5369 - val_accuracy: 0.7428 - val_loss: 0.5696
Epoch 6/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7669 - loss: 0.5008 - val_accuracy: 0.7460 - val_loss: 0.5711
Epoch 7/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7918 - loss: 0.4423 - val_accuracy: 0.7428 - val_loss: 0.5589
Epoch 8/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8135 - loss: 0.4175 - val_accuracy: 0.7556 - val_loss:

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_30 (Dense)                │ (None, 15)             │           105 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 40)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 21)             │           861 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 41)             │           902 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 4)              │           168 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,416 (36.79 KB)

 Trainable params: 3,138 (12.26 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,278 (24.53 KB)

epoch/accuracy,▁▁▁▁▂▃▃▄▅▅▆▅▆▆▆▇▇█▇▇██▇▇█
epoch/epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▆▅▅▄▄▄▃▃▃▃▂▂▂▁▁▁▁▁▁▂▁▁
epoch/val_accuracy,▁▂▁▁▃▃▃▃▅▅▆▄▅▇▆▇▇▇▇█▇█▇▇█
epoch/val_loss,█▇▇▆▅▆▅▅▄▃▃█▃▂▂▂▂▂▂▁▂▁▂▁▁
epoch/accuracy,0.95498
epoch/epoch,24
epoch/learning_rate,0.01
epoch/loss,0.10347
epoch/val_accuracy,0.94534


In [39]:
X_test_proc = feature_encoder.transform(X_test)
y_test_proc = target_encoder.transform(y_test)

y_pred = model(X_test_proc)
y_pred = np.argmax(y_pred, axis = 1)
sklearn.metrics.accuracy_score(y_pred, y_test_proc)

/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


0.9479768786127167

In [219]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

class Explainer:
    def __init__(self, model, X, preprocessor, n_samples = 100, rep_prob = 0.5):

        '''
        Args:
            model: model
            X: dataframe of X data (training, testing or all)
            preprocessor: function used to preprocess data prior to model forward pass
            n_samples: number of samples to generate for a single prediction explanation
            rep_prob: probability at which each feature value of x datapoint should be replaced with a value from a sample
        '''

        self.model = model
        self.X = X.to_numpy()
        self.preprocessor = preprocessor
        self.x_cols = X.columns
        self.y_col = 'target'
        self.n_samples = n_samples
        self.rep_prob = rep_prob
        
        self.data = None
        self.patterns = []
        self.relevance_dict = {col: 0 for col in self.x_cols}

    def __init_structures(self):
        self.data = None
        self.patterns = []
        
    def __data_generation(self, x: np.ndarray):

        y = self.model.predict(self.preprocessor(x.reshape(1, -1)), verbose = 0).squeeze(0)

        samples_X = []
        samples_Y = []
        
        for i in range(self.n_samples):
            
            sample_x = self.X[np.random.randint(len(self.X))]
            sample_x = np.where(np.random.rand(len(sample_x)) < self.rep_prob, x, sample_x)

            sample_y = self.model.predict(self.preprocessor(sample_x.reshape(1, -1)), verbose = 0).squeeze(0)
            
            sample_y = sample_y.argmax() != y.argmax()
            sample_x = sample_x != x
            
            samples_X.append(sample_x.astype(int))
            samples_Y.append(sample_y.astype(int))
            
        self.data = pd.DataFrame(samples_X, columns = self.x_cols)
        self.data[self.y_col] = samples_Y

    def fp_growth(self, data, class_):

        if class_ == 0:
            data = 1 - data
            
        data = data.astype(bool)

        class_patterns = fpgrowth(data, min_support = 0.3, use_colnames = True)
        
        self.patterns.append(class_patterns)

    def __harmonic_merge(self):

        self.patterns[0] = self.patterns[0].rename(columns = {'support': 'support_stable'})
        self.patterns[1] = self.patterns[1].rename(columns = {'support': 'support_unstable'})
        
        patterns_merged = pd.merge(
            self.patterns[0],
            self.patterns[1],
            on = 'itemsets',
            how = 'outer'
        )
        
        patterns_merged = patterns_merged.fillna(0)
        
        support_0 = patterns_merged['support_stable']
        support_1 = patterns_merged['support_unstable']

        denominator = support_0 + support_1
        patterns_merged['Harmonic Mean'] = (2 * support_0 * support_1 / denominator).mask(denominator == 0, 0)
        
        harmonic_rank = patterns_merged[['itemsets', 'support_stable', 'support_unstable', 'Harmonic Mean']].sort_values(
            by = 'Harmonic Mean', 
            ascending = False
        )
        
        return harmonic_rank[harmonic_rank['Harmonic Mean'] > 0.0]

    def __relevance_rank(self):

        rank = self.__harmonic_merge()

        for row, idx in rank.iterrows():
    
            features = tuple(idx['itemsets'])
            h_mean = idx['Harmonic Mean']
            n_features = len(features)
            
            for feature in features:
                self.relevance_dict[feature] += h_mean / n_features

    def forward(self, x) -> None:

        self.__init_structures()
        
        self.__data_generation(x)
        
        for class_ in [0, 1]:
            
            class_data = self.data.loc[self.data[self.y_col] == class_]
            class_data = class_data.drop(self.y_col, axis = 1)
            self.fp_growth(class_data, class_)
        
        self.__relevance_rank()

In [220]:
x_cols = car_evaluation.data.features.columns
y_col = car_evaluation.data.targets.columns[0]

def preprocessor(x):
    return feature_encoder.transform(x)

In [247]:
explainer = Explainer(
    model = model,
    X = car_evaluation.data.features,
    preprocessor = preprocessor,
    n_samples = 1000,
    rep_prob = 0.75
)

In [248]:
for _ in range(15):
    x = X_test[np.random.randint(len(X_test))]
    explainer.forward(x)

In [249]:
dict(sorted(explainer.relevance_dict.items(), key = lambda item: item[1], reverse = True))

{'persons': 9.878723206038899,
 'safety': 8.640487107730156,
 'buying': 4.865268068454181,
 'maint': 3.202407364106607,
 'lug_boot': 2.886463342673793,
 'doors': 1.7391728727259028}

In [ ]:
safety 
persons 
buying 
maint 
lug_boot 
doors